In [94]:
import numpy as np
import pandas as pd
import pickle
import tsplib95

In [80]:
def seed():
    np.random.seed(42)

In [42]:
def load_path(file_path):
    '''Load the optimal route.'''
    dic = tsplib95.load(file_path)
    return dic.tours[0]

optimal_route = load_path("TSP-Configurations\eil51.opt.tour.txt")
print(optimal_route)

def load_file(file_path): 
    '''Load the coordinates of each dimension.'''
    dic = tsplib95.load(file_path)

    x_coordinates = [coords[0] for _, coords in list(dic.node_coords.items())]
    y_coordinates = [coords[1] for _, coords in list(dic.node_coords.items())]
    
    return np.array(x_coordinates), np.array(y_coordinates)

x_coordinates, y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
print(x_coordinates)
print(y_coordinates)

[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
[37 49 52 20 40 21 17 31 52 51 42 31  5 12 36 52 27 17 13 57 62 42 16  8
  7 27 30 43 58 58 37 38 46 61 62 63 32 45 59  5 10 21  5 30 39 32 25 25
 48 56 30]
[52 49 64 26 30 47 63 62 33 21 41 32 25 42 16 41 23 33 13 58 42 57 57 52
 38 68 48 67 48 27 69 46 10 33 63 69 22 35 15  6 17 10 64 15 10 39 32 55
 28 37 40]


In [66]:
def get_distance(route, x_coordinates, y_coordinates): 
    '''Calculate the total distance with given route.'''
    distance = 0
    for idx, i in enumerate(route[1:]): 
        x_distance = np.abs(x_coordinates[i-1] - x_coordinates[route[idx]-1])
        y_distance = np.abs(y_coordinates[i-1] - y_coordinates[route[idx]-1])
        distance += np.sqrt(x_distance**2 + y_distance**2)
    x_distance =  np.abs(x_coordinates[route[-1]-1] - x_coordinates[route[0]-1])
    y_distance =  np.abs(y_coordinates[route[-1]-1] - y_coordinates[route[0]-1])
    distance += np.sqrt(x_distance**2 + y_distance**2)
    
    return distance

distance = get_distance(optimal_route, x_coordinates, y_coordinates)
print(optimal_route)
print(distance)


[1, 22, 8, 26, 31, 28, 3, 36, 35, 20, 2, 29, 21, 16, 50, 34, 30, 9, 49, 10, 39, 33, 45, 15, 44, 42, 40, 19, 41, 13, 25, 14, 24, 43, 7, 23, 48, 6, 27, 51, 46, 12, 47, 18, 4, 17, 37, 5, 38, 11, 32]
429.98331198338406


In [ ]:
def get_distance_matrix(x_coordinates, y_coordinates): 
    dimension = len(x_coordinates)
    distance_matrix = np.zeros((dimension, dimension))
    for i in range(dimension): 
        distance_matrix[i] = (x_coordinates - x_coordinates[i])**2
        distance_matrix[i] += (y_coordinates - y_coordinates[i])**2
    
    return np.sqrt(distance_matrix)

distance_matrix = get_distance_matrix(x_coordinates, y_coordinates)

In [67]:
optimal_route_ = (optimal_route - np.ones(51)).astype(int)
reshape_route = np.array((optimal_route_[:-1], optimal_route_[1:])).T
reshape_route = np.vstack((reshape_route, np.array([[optimal_route_[-1], optimal_route_[0]]])))
distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()

429.983311983384

In [29]:
rng = np.random.default_rng()
random_path = rng.choice(np.arange(1, 52), size=51, replace=False)

distance = get_distance(random_path, x_coordinates, y_coordinates)
print(distance)

1746.4935162039571


In [97]:
class SA(): 
    def __init__(self, dimension, temperature, num_i):
        if dimension == 51: 
            self.dimension = 51
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\eil51.tsp.txt")
        elif dimension == 280:
            self.dimension = 280
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\a280.tsp.txt")
        elif dimension == 442: 
            self.dimension = 442
            self.x_coordinates, self.y_coordinates = load_file("TSP-Configurations\pcb442.tsp.txt")
        else: 
            raise ValueError('No such file')
        
        rng = np.random.default_rng()
        self.route = rng.choice(range(self.dimension), size=self.dimension, replace=False)
        self.distance_matrix = self.get_distance_matrix()
        self.distance = self.get_distance(self.route)
        self.T = temperature
        self.coef_cooling_lin = temperature * (1 - 0.9 ** num_i) / num_i
        self.coef_cooling_log = np.log(2)
        self.num_i = num_i

    def get_distance_matrix(self): 
        distance_matrix = np.zeros((self.dimension, self.dimension))
        for i in range(self.dimension): 
            distance_matrix[i] = (x_coordinates - x_coordinates[i])**2
            distance_matrix[i] += (y_coordinates - y_coordinates[i])**2
        
        return np.sqrt(distance_matrix)
    
    def get_distance(self, route):
        reshape_route = np.array((route[:-1], route[1:])).T
        reshape_route = np.vstack((reshape_route, np.array([[route[-1], route[0]]])))

        return self.distance_matrix[reshape_route[:, 0], reshape_route[:, 1]].sum()
    
    def two_opt(self): 
        position_1, position_2 = np.sort(rng.choice(range(self.dimension), size=2, replace=False))
        new_route = np.copy(self.route)
        new_route[position_1:position_2+1] = self.route[np.arange(position_2, position_1-1, -1)]
        new_distance = self.get_distance(new_route) 

        return new_route, new_distance

    def acceptance_criteria_sa(self, new_route, new_distance): 
        '''acceptance criteria for simulated annealing algorithm'''
        random = np.random.rand()
        if random <= min(np.exp(-(new_distance-self.distance)/self.T),1): 
            self.route = new_route
            self.distance = new_distance

    def acceptance_criteria_hc(self, new_route, new_distance): 
        '''acceptance criteria for hill climbing algorithm'''
        if new_distance < self.distance:
            self.route = new_route
            self.distance = new_distance

    def cooling_schedule_lin(self): 
        # Balance the temperature after num_i iterations of linear and geometric cooling schedules
        self.T -= self.coef_cooling_lin

    def cooling_schedule_geo(self): 
        self.T *= 0.9

    def cooling_schedule_log(self): 
        self.current_iteration += 1
        self.T = self.coef_cooling_log / np.log(1+self.current_iteration)

    def run_simulation_sa_lin(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            self.cooling_schedule_lin()
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
    def run_simulation_sa_geo(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            self.cooling_schedule_geo()
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
    def run_simulation_sa_log(self): 
        distance_list = np.zeros(self.num_i)
        self.current_iteration = 0
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_sa(new_route, new_distance)
            self.cooling_schedule_log()
            distance_list[i] = self.distance
        
        return distance_list, self.route
    
    def run_simulation_hc(self): 
        distance_list = np.zeros(self.num_i)
        for i in range(self.num_i): 
            new_route, new_distance = self.two_opt()
            self.acceptance_criteria_hc(new_route, new_distance)
            distance_list[i] = self.distance
        
        return distance_list, self.route
    

In [98]:
def run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50): 
    
    seed()
    results = np.zeros((num_run, num_i))
    for i in range(num_run): 
        results[i,:], route = SA(dimension, temperature, num_i).run_simulation_sa_geo()

    return results, route
        
results, route = run_multiple_simulation(dimension=51, temperature=1, num_i=1000, num_run=50)

C:\Users\lishu\AppData\Local\Temp\ipykernel_23452\3763088304.py:49: RuntimeWarning: overflow encountered in exp
  if random <= min(np.exp(-(new_distance-self.distance)/self.T),1):


In [99]:
results

array([[1714.81362336, 1714.81362336, 1679.19030007, ...,  607.25738694,
         607.25738694,  607.25738694],
       [1712.95669585, 1712.95669585, 1712.95669585, ...,  583.02359429,
         583.02359429,  583.02359429],
       [1729.32529618, 1709.24253137, 1702.29503776, ...,  588.8512497 ,
         588.8512497 ,  588.8512497 ],
       ...,
       [1598.5899447 , 1592.41570979, 1592.41570979, ...,  581.16512316,
         581.16512316,  581.16512316],
       [1809.49247779, 1793.70210331, 1793.70210331, ...,  601.72313505,
         601.72313505,  601.72313505],
       [1816.67122347, 1816.67122347, 1816.67122347, ...,  594.04480756,
         594.04480756,  594.04480756]])